In [1]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import datetime
import time

from datetime import datetime

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import calendar
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from ipywidgets import interact, widgets
from matplotlib.dates import MonthLocator, DateFormatter
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month
from sklearn.metrics.pairwise import euclidean_distances
from pyspark.sql.functions import year, month, sum as sum_
from scipy.spatial.distance import euclidean
from prophet import Prophet
from pyspark.sql.functions import col, countDistinct , desc , count , length
from pyspark.sql.functions import split, col, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, dayofmonth, last_day ,to_date, lit, isnull
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import date_format

import schedule
from pyspark.sql.functions import col, to_timestamp, date_format

import pyspark
import pandas as pd
import boto3
import sagemaker
import sagemaker.feature_store.feature_store as fs
import databricks.connect
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go
from itertools import product
import random
import statsmodels.api as sm
import os

from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, months_between, current_date,expr

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from databricks.connect import DatabricksSession


# Get spark
spark = databricks.connect.DatabricksSession.builder.getOrCreate()

c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/14/25 08:44:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=926959;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=645576;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\gusta\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
query=  """WITH 

-- CTE para obter informações básicas dos clientes, como quantidade de contratos, primeiro e último bloqueio, motivo e status de conexão

clientes AS 
(
    SELECT 
          COUNT(1)              AS contratos          -- Contagem total de contratos por documento
        , cgcmf                 AS documento          -- Documento do cliente (CNPJ)
        , MIN(data_cad)         AS data_inicio        -- Data de início do primeiro contrato do cliente
        , MAX(bloqueio)         AS data_final         -- Data do último bloqueio do cliente
        , MAX_BY(max_conexao, bloqueio) AS max_conexao        -- Status de conexão mais recente
    FROM 
    (
        -- Seleção dos clientes únicos, eliminando duplicatas e aplicando filtros iniciais

        SELECT DISTINCT
            matriz, codigo, fis_jurid, cgcmf, raz_social, max_conexao, bloqueio, mot_bloqueio,
            sit_cliente, data_cad, vendedor, vlrfatmin, vlr_dispacesso, receita
        FROM 
            prd.dl_redeok_checkok.tb_cliente cli
        WHERE 
            cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')         -- Filtra clientes com status ativos ou pendentes
            AND (cli.matriz = cli.codigo OR falsa = 's')         -- Mantém apenas contas da matriz ou marcadas como falsas
            AND cli.receita IN (9, 10, 46, 58)                   -- Filtra por tipos de receita específico
            AND cli.cgcmf IS NOT NULL                            -- Remove registros sem CNPJ
            AND cli.cgcmf != '23236392000198'                    -- Exclui um CNPJ específico (ROK)
            AND LENGTH(TRIM(cli.cgcmf)) = 14                     -- Garante que o CNPJ tenha 14 caracteres
            AND cli.cgcmf RLIKE '^[0-9]+$'                       -- Garante que o CNPJ seja numérico

            -- Remove registros que fazem parte da tabela de revendas

            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE cli.codigo = rev.codigo
            )
    ) 
    GROUP BY cgcmf
),

-- CTE para contar a quantidade de boletos gerados por cliente

titulo_cobranca AS 
(
    SELECT 
          documento         AS documento                
        , cliente          AS cliente  
        , COUNT(meses)      AS boletos_gerados        -- Contagem de meses com boletos gerados
    FROM 
    (

    -- Seleção de boletos distintos por cliente e mês

        SELECT DISTINCT
            cli.cgcmf                           AS documento
            , rec.cliente                       AS cliente  
            , DATE_TRUNC('month', rec.emissao)  AS meses     -- Agrupa por mês de emissão do boleto
        FROM 
            prd.dl_redeok_checkok.vw_receber rec
        INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
            ON cli.codigo = rec.cliente
        WHERE 
            rec.situacao IN ('A', 'B')

             -- Remove registros de revendas

            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE rev.codigo = rec.cliente
            )

             -- Filtra os mesmos clientes da CTE 'clientes'
             
            AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14    
            AND cli.cgcmf RLIKE '^[0-9]+$'
    ) 
    GROUP BY 
        documento, cliente  
),

-- CTE para encontrar a data do último boleto gerado por cliente

ultimo_boleto AS 
(
    SELECT 
        cli.cgcmf AS documento,
        MAX(rec.emissao) AS ultima_emissao            -- Última data de emissão do boleto
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')                   -- Apenas boletos ativos ou em aberto

        -- Remove registros de revendas

        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
),

-- CTE para encontrar a data do primeiro boleto gerado por cliente

primeiro_boleto AS 
(
    SELECT 
        cli.cgcmf AS documento,
        MIN(rec.emissao) AS data_primeiro_boleto
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')
        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
),

-- CTE para somar o total pago por cliente

soma_valor_pago AS
(
    SELECT 
        cli.cgcmf AS documento,
        SUM(rec.valor_pago) AS total_valor_pago       -- Soma total de valores pagos pelo cliente
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')

        -- Remove registros de revendas
        
        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
),
documentos_filtrados AS (
    SELECT 
          cli.contratos
        , cli.documento
        , tit.cliente  
        , cli.data_inicio
        , cli.data_final  
        , cli.max_conexao 
        , prim.data_primeiro_boleto   
        , CASE 
              WHEN cli.data_final IS NULL THEN NULL
              ELSE ult.ultima_emissao
          END AS data_ultimo_boleto
        , tit.boletos_gerados
        , ROUND(MONTHS_BETWEEN(COALESCE(cli.data_final, CURRENT_DATE()), cli.data_inicio)) AS meses_contrato
        , ROUND(COALESCE(soma.total_valor_pago, 0), 1) AS total_valor_pago
        , ROW_NUMBER() OVER (PARTITION BY cli.documento ORDER BY tit.boletos_gerados DESC) AS row_number
     FROM 
        clientes cli
    INNER JOIN titulo_cobranca tit 
        ON cli.documento = tit.documento
    LEFT JOIN ultimo_boleto ult
        ON cli.documento = ult.documento
    LEFT JOIN primeiro_boleto prim
        ON cli.documento = prim.documento
    LEFT JOIN soma_valor_pago soma
        ON cli.documento = soma.documento
)
SELECT 
    contratos,
    documento,
    cliente,
    data_inicio,
    data_final,
    max_conexao,
    data_primeiro_boleto,
    data_ultimo_boleto,
    boletos_gerados,
    meses_contrato,
    total_valor_pago
FROM documentos_filtrados
WHERE row_number = 1;

"""


# Executando a consulta e obtendo os dados em um DataFrame Spark
tabela_nova_sdf= spark.sql(query)

from pyspark.sql.functions import round

# Criar a coluna 'tipo_saida' com as condições especificadas
tabela_nova_sdf = tabela_nova_sdf.withColumn(
    "tipo_saida",
    F.when(F.col("max_conexao") == -1, "CANCELAMENTO NORMAL")
    .when(F.col("max_conexao") == 0, "90 DIAS BLOQUEADO")
    .when(F.col("max_conexao") == -5, "90 DIAS PRÉ CANCELADO")
    .when(F.col("max_conexao") == -7, "90 DIAS PRÉ CANCELADO JURÍDICO")
    .when(F.col("max_conexao").isNull(), "ATIVO")
    .otherwise("OUTROS")
)

tabela_nova_sdf = tabela_nova_sdf.withColumn("anos_boletos_gerados", round(col("boletos_gerados") / 12, 2))


from pyspark.sql.functions import col, to_date

from pyspark.sql.functions import col, to_timestamp

# Converter primeiro para timestamp
#tabela_nova_sdf = tabela_nova_sdf.withColumn("data_primeiro_boleto", to_timestamp(col("data_primeiro_boleto"), "yyyy-MM-dd HH:mm:ss"))

# Depois converter para date
#tabela_nova_sdf = tabela_nova_sdf.withColumn("data_primeiro_boleto", col("data_primeiro_boleto").cast("date"))


# Criar a coluna 'anos_boleto'
tabela_nova_sdf = tabela_nova_sdf.withColumn(
    "anos_boleto",
    F.round(
        F.months_between(
            F.coalesce("data_ultimo_boleto", F.current_date()),  # Se for nulo, usa a data atual
            "data_primeiro_boleto"
        ) / 12, 2  # Divide por 12 para converter para anos e arredonda para 2 casas decimais
    )
)
tabela_nova_sdf = tabela_nova_sdf.withColumn("anos_contrato", round(col("meses_contrato") / 12, 2))

tabela_nova_sdf = tabela_nova_sdf.withColumn("churn",when(col("data_final").isNotNull(), 1).otherwise(0))

tabela_nova_sdf = tabela_nova_sdf.filter((col("meses_contrato") > 0))

tabela_nova_sdf = tabela_nova_sdf.filter(col("contratos") == 1)

# Substituir valores nulos pela data atual
# Criar colunas corrigindo a diferença para ser sempre positiva
tabela_nova_sdf = tabela_nova_sdf.withColumn(
    "diferenca_meses_inicio_primeiro_boleto",
    expr("ceil(abs(months_between(data_primeiro_boleto, data_inicio)))")
).withColumn(
    "diferenca_meses_final_ultimo_boleto",
    expr("abs(floor(months_between(COALESCE(data_final, current_date()), COALESCE(data_ultimo_boleto, current_date()))))")
)

from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, expr

# Criar a classificação para diferença de meses
tabela_nova_sdf = tabela_nova_sdf.withColumn(
    "classifica_diferenca_boleto_tempo_inicial",
    when(col("diferenca_meses_inicio_primeiro_boleto") <= 1, "0 a 1 mês")
    .when((col("diferenca_meses_inicio_primeiro_boleto") >= 2) & (col("diferenca_meses_inicio_primeiro_boleto") <= 5), "2 a 5 meses")
    .when((col("diferenca_meses_inicio_primeiro_boleto") >= 6) & (col("diferenca_meses_inicio_primeiro_boleto") <= 12), "6 a 12 meses")
    .when((col("diferenca_meses_inicio_primeiro_boleto") > 12) & (col("diferenca_meses_inicio_primeiro_boleto") <= 24), "13 a 24 meses")
    .otherwise("Maior que 24 meses")
).withColumn(
    "classifica_diferenca_boleto_tempo_final",
    when(col("diferenca_meses_final_ultimo_boleto") <= 1, "0 a 1 mês")
    .when((col("diferenca_meses_final_ultimo_boleto") >= 2) & (col("diferenca_meses_final_ultimo_boleto") <= 5), "2 a 5 meses")
    .when((col("diferenca_meses_final_ultimo_boleto") >= 6) & (col("diferenca_meses_final_ultimo_boleto") <= 12), "6 a 12 meses")
    .when((col("diferenca_meses_final_ultimo_boleto") > 12) & (col("diferenca_meses_final_ultimo_boleto") <= 24), "13 a 24 meses")
    .otherwise("Maior que 24 meses")
)




In [3]:
churn_sdf = spark.table("dev.dw_redeok_model.tb_f_churn")
churn_sdf_cliente = spark.table("dev.dw_redeok_model.tb_d_customer")

In [4]:
from pyspark.sql.functions import col

# Realizando o left join
churn_sdf_final = churn_sdf.join(
    churn_sdf_cliente,
    churn_sdf["uuid_customer"] == churn_sdf_cliente["uuid"],
    "left"
)

# Selecionando apenas as colunas desejadas
churn_sdf_final = churn_sdf_final.select(
    col("uuid_contract"),
    col("uuid_customer"),
    col("churn_date"),
    col("churn_type"),
    col("cnpj_cpf")
)


from pyspark.sql.functions import col

# Realizando o left join entre churn_sdf_final e tabela_nova_sdf
churn_sdf_compara = churn_sdf_final.join(
    tabela_nova_sdf,
    churn_sdf_final["cnpj_cpf"] == tabela_nova_sdf["documento"],
    "left"
)

# Selecionando apenas as colunas desejadas
churn_sdf_compara = churn_sdf_compara.select(
    col("uuid_contract"),
    col("uuid_customer"),
    col("churn_date"),
    col("churn_type"),
    col("cnpj_cpf"),
    col("tipo_saida"),
    col("documento"),
    col("contratos")
)

# Exibir resultado
churn_sdf_compara.show()
churn_sdf_compara.count()

+--------------------+--------------------+-------------------+--------------------+--------------+-------------------+--------------+---------+
|       uuid_contract|       uuid_customer|         churn_date|          churn_type|      cnpj_cpf|         tipo_saida|     documento|contratos|
+--------------------+--------------------+-------------------+--------------------+--------------+-------------------+--------------+---------+
|5774a775-bf8d-478...|3e03da5e-eed1-461...|2024-05-28 14:59:24| CANCELAMENTO NORMAL|05509938000155|CANCELAMENTO NORMAL|05509938000155|        1|
|3e4f91a3-d961-419...|3e03da5e-eed1-461...|2024-05-28 14:36:31| CANCELAMENTO NORMAL|05509938000155|CANCELAMENTO NORMAL|05509938000155|        1|
|5bce6f1c-7f78-402...|cb4c1bc9-2bb8-4d7...|2024-09-16 10:51:32| CANCELAMENTO NORMAL|15175658000101|CANCELAMENTO NORMAL|15175658000101|        1|
|81e94a75-e78b-4ed...|9748c73f-6bfc-4cc...|2024-09-16 10:48:30| CANCELAMENTO NORMAL|04514487000181|CANCELAMENTO NORMAL|04514487000

24296

In [5]:
from pyspark.sql.functions import col

# Filtrar registros onde churn_type e tipo_saida são diferentes, considerando casos onde uma delas é nula
diferentes_2 = churn_sdf_compara.filter(
    (col("churn_type") != col("tipo_saida")) |
    (col("churn_type").isNull() & col("tipo_saida").isNotNull()) |
    (col("churn_type").isNotNull() & col("tipo_saida").isNull())
)

# Contar quantos registros são diferentes
num_diferentes_x = diferentes_2.count()

print(f"Total de registros onde churn_type e tipo_saida são diferentes: {num_diferentes_x}")

diferentes_2.show(5000,truncate=False)

Total de registros onde churn_type e tipo_saida são diferentes: 5879
+------------------------------------+------------------------------------+-------------------+------------------------------+--------------+------------------------------+--------------+---------+
|uuid_contract                       |uuid_customer                       |churn_date         |churn_type                    |cnpj_cpf      |tipo_saida                    |documento     |contratos|
+------------------------------------+------------------------------------+-------------------+------------------------------+--------------+------------------------------+--------------+---------+
|40a2cccb-658f-4775-8476-cd42598b5690|9899d76b-0022-459d-9a9d-cb057eec4395|2021-06-30 17:30:16|CANCELAMENTO NORMAL           |10944409000110|NULL                          |NULL          |NULL     |
|b296b80a-50ab-49ad-8821-c3ed72868ceb|29594b50-fbee-485e-97a6-d6f57690ace8|2021-08-20 12:06:05|CANCELAMENTO NORMAL           |1718026300012

In [6]:
from pyspark.sql import functions as F

# Total de registros na base completa de churn
total_clientes = churn_sdf_compara.count()

# Total de registros na base de diferenças
total_diferencas = diferentes_2.count()

# Contagem das diferenças por churn_type e tipo_saida
contagem_diferencas = diferentes_2.groupBy("churn_type", "tipo_saida").count()

# Adicionando porcentagem baseada na base completa de churn
contagem_diferencas = contagem_diferencas.withColumn(
    "percent_total",
    F.round((F.col("count") / total_clientes * 100), 2))


# Ordenar e exibir
contagem_diferencas = contagem_diferencas.orderBy(F.col("churn_type").asc())
contagem_diferencas.show(100, truncate=False)

# Soma total das colunas count e percent_total
soma_totais = contagem_diferencas.agg(
    F.sum("count").alias("total_count"),
    F.sum("percent_total").alias("total_percent")
)

# Mostrar o resultado
soma_totais.show()

+------------------------------+------------------------------+-----+-------------+
|churn_type                    |tipo_saida                    |count|percent_total|
+------------------------------+------------------------------+-----+-------------+
|90 DIAS BLOQUEADO             |ATIVO                         |312  |1.28         |
|90 DIAS BLOQUEADO             |NULL                          |1069 |4.4          |
|90 DIAS BLOQUEADO             |CANCELAMENTO NORMAL           |241  |0.99         |
|90 DIAS BLOQUEADO             |OUTROS                        |6    |0.02         |
|90 DIAS BLOQUEADO             |90 DIAS PRÉ CANCELADO         |22   |0.09         |
|90 DIAS PRÉ CANCELADO         |CANCELAMENTO NORMAL           |6    |0.02         |
|90 DIAS PRÉ CANCELADO         |90 DIAS BLOQUEADO             |74   |0.3          |
|90 DIAS PRÉ CANCELADO         |NULL                          |33   |0.14         |
|90 DIAS PRÉ CANCELADO         |90 DIAS PRÉ CANCELADO JURÍDICO|1    |0.0    

**Quantificação casos que não estão batendo:**
- A tabela mostra os valores inconsistentes, que era esperados para churn_type e os valores que estão realmente atribuídos na minha base (tipo_saida), destacando o total de inconsistências. Também exibe quantos clientes estão com dados inconsistentes e qual a porcentagem que isso representa no total.

**De forma geral, as inconsistências mais significativas são:**
- Clientes que deveriam estar como CANCELAMENTO NORMAL, mas aparecem como NULL na minha base (9,8%).
- Clientes que deveriam ser CANCELAMENTO NORMAL, mas estão registrados como 90 DIAS BLOQUEADO (6,22%).
Clientes que deveriam ser 90 DIAS BLOQUEADO, mas aparecem como NULL (4,4%).
- Quase 15% das inconsistências são de casos onde o CNPJ não foi encontrado na minha tabela (NULL)
- A maioria dos registros está consistente (76%), mas há diferenças que podem ser explicadas pela exclusão de clientes com mais de um contrato, clientes com datas irregulares excluídas e a forma como os contratos são armazenados na tabela oficial(clientes com mais de um contrato na tabela churn).A